## import libraries

In [26]:
import os
from glob import glob
import pandas as pd
from functools import reduce
import xml.etree.ElementTree as ET
ImportWarning

ImportWarning

In [27]:
# Load all xml files and store in a list
xml_list=glob(r"C:\Users\Syed Sajad\Desktop\Data_preparation\data_images\*.xml")

In [28]:
xml_list

['C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\1.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\2.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115451_jpg.rf.77d682ffd085a616a8a82b113ee2abfc.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115459_jpg.rf.6e1df96fbc32d039434cfba3efd776d7.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115505_jpg.rf.ce05334ac3b69d425cbfadb0efbbec62.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115513_jpg.rf.23a2f2771e36c785b4cfb0cf2e7ade21.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115521_jpg.rf.ac95027de1735594dbcd81a5a7812819.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115527_jpg.rf.0408cff74f04f00304d22538e94cd821.xml',
 'C:\\Users\\Syed Sajad\\Desktop\\Data_preparation\\data_images\\20190809_115534_jpg.rf.110b

In [29]:
tree=ET.parse('data_images/IMG20200730132847_jpg.rf.11b4d1edfec99016257d84a5efc2ba21.xml')
root=tree.getroot()
objs=root.findall('object')
obj=objs[0]
name = obj.find('name').text 
bndbox = obj.find('bndbox')
xmin = bndbox.find('xmin').text
xmax = bndbox.find('xmax').text
ymin = bndbox.find('ymin').text
ymax = bndbox.find('ymax').text
[name,xmin,xmax,ymin,ymax]

['bad banana', '29', '161', '56', '121']

In [30]:
# Read xml files 
# from each xml files we need to extract
# file name, size(height and width), object(name, xmin, xmax, ymin, ymax)

def extract_text(filename):
    tree = ET.parse(filename)
    root = tree.getroot()

    # Extract filename
    image_name = root.find('filename').text
    
    # Extract width and height of image
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    # Extract object information
    objects = root.findall('object')
    parser = []
    
    for obj in objects:
        name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        xmax = int(bbox.find('xmax').text)
        ymin = int(bbox.find('ymin').text)
        ymax = int(bbox.find('ymax').text)
        parser.append([image_name, width, height, name,xmin, xmax, ymin, ymax])
    
    return parser


In [31]:
parser_all=list(map(extract_text,xml_list))

In [32]:
data=reduce(lambda x, y : x+y, parser_all)
# converted into dataframe
data=pd.DataFrame(data, columns=['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [33]:
data.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,1.jpg,320,320,good apple,73,254,38,231
1,2.jpg,320,320,good apple,66,264,41,244
2,20190809_115451_jpg.rf.77d682ffd085a616a8a82b1...,320,320,good apple,57,245,42,254
3,20190809_115459_jpg.rf.6e1df96fbc32d039434cfba...,320,320,bad apple,60,201,91,229
4,20190809_115505_jpg.rf.ce05334ac3b69d425cbfadb...,320,320,bad apple,77,193,116,225


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7313 entries, 0 to 7312
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  7313 non-null   object
 1   width     7313 non-null   int64 
 2   height    7313 non-null   int64 
 3   name      7313 non-null   object
 4   xmin      7313 non-null   int64 
 5   xmax      7313 non-null   int64 
 6   ymin      7313 non-null   int64 
 7   ymax      7313 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 457.2+ KB


In [35]:
data['name'].value_counts()

name
good orange    1674
good banana    1594
bad orange     1336
good apple     1169
bad banana     1008
bad apple       532
Name: count, dtype: int64

## prepare labels for Yolo model

In [36]:
# find center x and center y
data['center_x']= ((data['xmax']+ data['xmin'])/2)/ data['width']
data['center_y']= ((data['ymax']+ data['ymin'])/2)/ data['height']

# width
data['w']= (data['xmax']-data['xmin'])/data['width']
# height
data['h']= (data['ymax']-data['ymin'])/data['height']

In [37]:
data.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,1.jpg,320,320,good apple,73,254,38,231,0.510938,0.420312,0.565625,0.603125
1,2.jpg,320,320,good apple,66,264,41,244,0.515625,0.445312,0.618750,0.634375
2,20190809_115451_jpg.rf.77d682ffd085a616a8a82b1...,320,320,good apple,57,245,42,254,0.471875,0.462500,0.587500,0.662500
3,20190809_115459_jpg.rf.6e1df96fbc32d039434cfba...,320,320,bad apple,60,201,91,229,0.407813,0.500000,0.440625,0.431250
4,20190809_115505_jpg.rf.ce05334ac3b69d425cbfadb...,320,320,bad apple,77,193,116,225,0.421875,0.532813,0.362500,0.340625


## Split Data into Train and Test

In [38]:
# check the images
images=data['filename'].unique()

In [39]:
len(images)

3299

In [40]:
# 80% train and 20% test
img_df=pd.DataFrame(images, columns=['filename'])
img_train=tuple(img_df.sample(frac=0.8)['filename']) # Shuffle and pick 80% of images
img_train

('IMG_2404_JPG_jpg.rf.6fe1435591088b348cb04a6f0713631d.jpg',
 'IMG20200728181208_jpg.rf.dcb320518a06110953d244eb0770e83c.jpg',
 'IMG_8006_JPG_jpg.rf.5bf9e4637733eed879362df8347b3a27.jpg',
 'IMG20200729190236_01_jpg.rf.8195d40fad5181d83e44f3bdac46efba.jpg',
 'IMG_8095_JPG_jpg.rf.ef4a8caa9551b34773beb2d490fcec4c.jpg',
 'IMG20200729183107_01_jpg.rf.1d4032439a4b0c6cf421a10e64c63919.jpg',
 'IMG_8586_JPG_jpg.rf.f8e1eccac42d2a575438e612305818d3.jpg',
 'IMG20200730132248_jpg.rf.44ae11ee493eedcd12952a15150efbf8.jpg',
 'IMG_0121_JPG_jpg.rf.c745e69f50a5e41261f982ba624fd84e.jpg',
 'IMG_0112_JPG_jpg.rf.05586279a4d1f131de0f9da211e0da34.jpg',
 'IMG_2367_JPG_jpg.rf.0005158a5cff766d47e5bafc17cde2cb.jpg',
 'IMG_20190924_152402_jpg.rf.900338a80b1ec0230009df774cc79f2e.jpg',
 '20190809_120333_jpg.rf.bc07f74c52abbea7ada792fca0dc906c.jpg',
 'IMG_8193_JPG_jpg.rf.31b7543728f3c8b32fb9ccca170958d4.jpg',
 'IMG_8143_JPG_jpg.rf.b8c188ed6e34b16b80ee6bee479b16c8.jpg',
 'IMG20200728161548_jpg.rf.e31155119e44e9697afd8b

In [41]:
img_test=tuple(img_df.query(f'filename not in {img_train}')['filename'])
img_test

('20190809_115534_jpg.rf.110be304cec5726de908e4048b6ad24a.jpg',
 '20190809_115717_jpg.rf.d369fffedad47cf6cd56bb31bffc505c.jpg',
 '20190809_115723_jpg.rf.de7102654bf3bf668b9349b484d7f447.jpg',
 '20190809_115733_jpg.rf.d134b6eda3233a1f75c450edf6aed894.jpg',
 '20190809_115758_jpg.rf.419734c72eb5a701965f0d103d00d779.jpg',
 '20190809_115815_jpg.rf.ef38a0a8ce761e14aa2be26856035d23.jpg',
 '20190809_120034_jpg.rf.e34264de2a62b3a32ecd384387a45e33.jpg',
 '20190809_120105_jpg.rf.aefa0b63391c0bab3271ee078d96ffb8.jpg',
 '20190809_120123_jpg.rf.ca807bd3f513a394c35e0727deed4448.jpg',
 '20190809_120222_jpg.rf.32c5cbe0aee3be026d218e59e8b2d368.jpg',
 '20190809_120239_jpg.rf.16f87869d197c3eede936086360f07ac.jpg',
 '20190809_120328_jpg.rf.06932695a23487dcca4b74603348ca77.jpg',
 '20190809_120445_jpg.rf.a0cd4967f6a15925fda925b7505d5b59.jpg',
 '20190809_120459_jpg.rf.4b533a5d6a4537120217676787f0e88c.jpg',
 '20190809_120516_jpg.rf.9997d756fb4ba5f67630d4edce44721e.jpg',
 '20190809_120648_jpg.rf.b577b534d94ccaf

In [42]:
len(img_train), len(img_test)

(2639, 660)

In [43]:
train_df=data.query(f'filename in {img_train}')
test_df=data.query(f'filename in {img_test}')

In [44]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,1.jpg,320,320,good apple,73,254,38,231,0.510938,0.420312,0.565625,0.603125
1,2.jpg,320,320,good apple,66,264,41,244,0.515625,0.445312,0.618750,0.634375
2,20190809_115451_jpg.rf.77d682ffd085a616a8a82b1...,320,320,good apple,57,245,42,254,0.471875,0.462500,0.587500,0.662500
3,20190809_115459_jpg.rf.6e1df96fbc32d039434cfba...,320,320,bad apple,60,201,91,229,0.407813,0.500000,0.440625,0.431250
4,20190809_115505_jpg.rf.ce05334ac3b69d425cbfadb...,320,320,bad apple,77,193,116,225,0.421875,0.532813,0.362500,0.340625


In [45]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
8,20190809_115534_jpg.rf.110be304cec5726de908e40...,320,320,bad apple,56,253,81,295,0.482812,0.587500,0.615625,0.668750
18,20190809_115717_jpg.rf.d369fffedad47cf6cd56bb3...,320,320,good apple,84,239,70,216,0.504687,0.446875,0.484375,0.456250
19,20190809_115723_jpg.rf.de7102654bf3bf668b9349b...,320,320,good apple,70,221,62,227,0.454688,0.451562,0.471875,0.515625
20,20190809_115733_jpg.rf.d134b6eda3233a1f75c450e...,320,320,good apple,113,245,86,212,0.559375,0.465625,0.412500,0.393750
22,20190809_115758_jpg.rf.419734c72eb5a701965f0d1...,320,320,good apple,92,254,101,256,0.540625,0.557813,0.506250,0.484375


In [46]:
# label Encoding of the objects
def label_encoding(x):
    labels={'good orange':0,'good banana':1,'bad orange':2,
            'good apple':3,'bad banana':4,'bad apple':5 } 
    return labels[x]   

In [ ]:
train_df['id']=train_df['name'].apply(label_encoding)
test_df['id']=test_df['name'].apply(label_encoding)

In [48]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,1.jpg,320,320,good apple,73,254,38,231,0.510938,0.420312,0.565625,0.603125,3
1,2.jpg,320,320,good apple,66,264,41,244,0.515625,0.445312,0.618750,0.634375,3
2,20190809_115451_jpg.rf.77d682ffd085a616a8a82b1...,320,320,good apple,57,245,42,254,0.471875,0.462500,0.587500,0.662500,3
3,20190809_115459_jpg.rf.6e1df96fbc32d039434cfba...,320,320,bad apple,60,201,91,229,0.407813,0.500000,0.440625,0.431250,5
4,20190809_115505_jpg.rf.ce05334ac3b69d425cbfadb...,320,320,bad apple,77,193,116,225,0.421875,0.532813,0.362500,0.340625,5


## save images and labels in text And seprate folders of train and test images

In [49]:
# import libraries
import os
from shutil import move

In [58]:
train_folder='data_images/train'
test_folder='data_images/test'

# os.mkdir(train_folder)
# os.mkdir(test_folder)

In [52]:
cols=['filename','id','center_x','center_y','w','h']
groupby_object_train=train_df[cols].groupby('filename')
groupby_object_test=test_df[cols].groupby('filename')

In [53]:
# save each image of train and test in folder and respective labels in .txt
def save_data(filename,folder_path, group_obj):
    # move images
    src=os.path.join('data_images',filename)
    dst=os.path.join(folder_path,filename)
    move(src,dst) # move images to destination folder

    # save the labels
    text_filename=os.path.join(folder_path,os.path.splitext(filename)[0]+'.txt')

    group_obj.get_group(filename).set_index('filename').to_csv(text_filename, sep=' ', index=False,header=False)

In [56]:
filename_series=pd.Series(groupby_object_train.groups.keys())

In [ ]:
filename_series

In [ ]:
filename_series.apply(save_data,args=(train_folder,groupby_object_train))

In [61]:
# now for test images 
filename_series_test=pd.Series(groupby_object_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_object_test))

0      None
1      None
2      None
3      None
4      None
       ... 
655    None
656    None
657    None
658    None
659    None
Length: 660, dtype: object